In [1]:
from scripts.software_frs import frs_traceability2
FRS = frs_traceability2
from officelib.xllib import *
from officelib.const import xlconst as xlc
import sys
import os
import re

from scripts.tools.issuetracker import IssuetrackerAPI
from datetime import datetime

In [2]:
_cache = None
_age = None
def _download_issues():
    api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
    return api.download_issues("pbssoftware")

def get_issues():
    global _cache, _age
    if _cache == None or (datetime.now() - _age).total_seconds() > (8*3600):  # 8 hr
        _cache = _download_issues()
        _age = datetime.now()
    return _cache

In [3]:
_item_match = re.compile(r"^\>?[\*]*\s*[\+\*]+(FRS[\d\.]+)\:?[\+\*]*\:?\s*(.*)$").match
_canceled_match = re.compile(r"^\>?[\*]*\s*-[\+\*]+(FRS[\d\.]+)\:?[\+\*]*\:?\s*(.*)-$").match
_header_match = re.compile(r"^\>?[\+\*]{2}([\d\.\w]+)[\+\*]{2}").match

def _extract_frs_lines(lines):
    start_matches = (
        (_item_match, 0),
        (_canceled_match, 1),
    )
    end_matches = (
        _item_match,
        _canceled_match,
    )
    i = 0
    while True:
        try:
            line = lines[i]
        except IndexError:
            break
        i += 1
        for match, flags in start_matches:
            done = False
            if not line:
                m = None
            else:
                m = match(line)
            if m:
                f = m.group(1)
                t = m.group(2).strip()
                l = [t]
                while True:
                    try:
                        line = lines[i].strip()
                    except IndexError:
                        done = True
                        break
                    if not line:
                        done = True
                        break
                    if any(m(line) for m in end_matches):
                        done = True
                        break
                    l.append(" " + line.strip())
                    i += 1
                text = "\n".join(l)
                yield f, flags, text
            if done:
                break          

In [4]:
def filter_relevant_issues(issues, rel_cb):
    relevant = []
    for v in issues.values():
        if rel_cb(v):
                relevant.append(v)
    return relevant

def load_frs_from_issuetracker(rel_cb):
    issues = get_issues()
    relevant = filter_relevant_issues(issues, rel_cb)
    
    all_frs = {None: (0, "")}
    for v in relevant:
        lines = v.description.splitlines()
        for f, flags, text in _extract_frs_lines(lines):
            all_frs[f] = flags, text
    del all_frs[None]
    return all_frs, relevant

def ws_data(ws):
    cr = ws.Cells.Range
    first = cr("A2")
    last_col = cr("A1").End(xlc.xlToRight).Column
    last = first.End(xlc.xlDown).Offset(1, last_col)
    return cr(first, last).Value

def split_frs(frs):
    return [f for f in frs.split() if f]

def find_cols(ws, *cols):
    first = ws.Cells(1,1)
    last = first.End(xlc.xlToRight)
    headers = ws.Cells.Range(first,last).Value[0]
    res = [headers.index(c) for c in cols]
    return res

def load_test_list(user_tests):
    xl = Excel(visible=False)
    with HiddenXl(xl):
        wb = xl.Workbooks.Open(user_tests)
        ws = wb.Worksheets(1)
        id_test, name, purpose, start, steps, accept, frs = \
                        find_cols(ws, 
                            "ID_TEST", "NAME", "PURPOSE", "STARTING POINT", 
                            "STEPS", "ACCEPTANCE_CRITERIA", "List Web FRS")
        data = ws_data(ws)
    
    wb.Close(False)
    xl.Quit()
    
    res = []
    for row in data:
        i = row[id_test]
        n = row[name]
        p = row[purpose]
        sta = row[start]
        ste = row[steps]
        a = row[accept]
        f = row[frs]
        if not f: continue
        res.append((i, n, p, sta, ste, a, split_frs(f)))
    res.sort(key=lambda t: t[0])
    return res

In [5]:
def paste_data(ws, data):
    cells = ws.Cells
    cr = cells.Range
    
    di = 2
    hi = 3
    
    header_start = cr("A1")
    
    frs_start = cr("A2")
    frs_end = header_start.Offset(len(data), 1)

    id_start = frs_start.Offset(1, 3)
    id_end = frs_end.Offset(1, 3)

    tested_start = id_start.Offset(1,2)
    tested_end = id_end.Offset(1,2)

    paste_start = header_start
    paste_end = frs_end.Offset(1, len(data[0]))
    
    holdup_end = tested_end.Offset(1, hi)
    
    xl = ws.Application
    
    with screen_lock(xl):
        paste_range = cr(paste_start, paste_end)
        print("Pasting test data")
        paste_range.Clear()
        #paste_range.NumberFormat = "@"
        paste_range.Value = data

        print("Applying alignment formatting")
        cr(frs_start, id_end).VerticalAlignment = xlc.xlTop
        cr(id_start, id_end).HorizontalAlignment = xlc.xlRight
        cr(id_start, id_end).NumberFormat = "@"
        cr(tested_start, holdup_end).HorizontalAlignment = xlc.xlRight
        cr(tested_start, holdup_end).VerticalAlignment = xlc.xlTop
        cr(tested_start.Offset(1, 3), paste_end).HorizontalAlignment=xlc.xlLeft

        print("Marking untested cells")
        cond = cr(header_start.Offset(2, 1),paste_end).FormatConditions.Add(Type=xlc.xlExpression, Formula1="=$C1=\"\"")
        rint = cond.Interior
        rint.PatternColorIndex = xlc.xlAutomatic
        rint.ThemeColor = xlc.xlThemeColorAccent2
        rint.TintAndShade = 0.399975585192419
        
        print("Applying row formatting")
        for d, row in zip(data, paste_range.Rows):
            count = d[0].count(".") * 2
            row.Cells(1,1).IndentLevel = count
            row.Cells(1,2).IndentLevel = count
            if count == 0:
                rint = row.Interior
                rint.Pattern = xlc.xlSolid
                rint.PatternColorIndex = xlc.xlAutomatic
                rint.ThemeColor = xlc.xlThemeColorAccent6
                rint.TintAndShade = 0.6
                
        rint = paste_range.Rows(1).Interior
        rint.Pattern = xlc.xlSolid
        rint.PatternColorIndex = xlc.xlAutomatic
        rint.ThemeColor = xlc.xlThemeColorDark1
        rint.TintAndShade = -0.249977111117893
                
        print("Applying column autofit")
        # fit after filter to account for width of filter icon
        paste_range.Columns(1).AutoFit()
        paste_range.Columns(2).AutoFit()
        
        print("Applying row autofit")
        for r in paste_range.Rows:
            r.EntireRow.AutoFit()

def make_paste_data(root):
    data = [("FRS Number", "Text", "Test IDs")]

    for node in root.iter():
        f = node.id
        tests = ";".join(str(i) for i in node.get_tests())
        if not tests and node.has_children():
            tests = "'--"
        text = node.text or ""

        data.append((f, text, tests))
    return data

In [6]:
from officelib.xllib import *


def build_frs_tree(all_items, issues):
    root = FRS.Root(FRS._key_match)
    for frs, (flags, text) in all_items.items():
        try:
            node = root.add(frs, flags)
        except:
            print(frs, flags, text)
            raise
        if not text and frs.count(".") == 0:
            text = issues[int(frs[3:])].subject
        node.text = text
    return root

def main(user_tests, rel_cb):
    all_frs_items, issues = load_frs_from_issuetracker(rel_cb)
    issues = {issue.id: issue for issue in issues}
    root = build_frs_tree(all_frs_items, issues)
    xl = Excel()
    with screen_lock(xl):
        print("Loading User Test Matrix...")
        FRS.process_user_tests(xl, root, user_tests)

    print("Compiling data for final matrix")
    data = make_paste_data(root)
    
    ws = FRS.get_matrix_sheet(xl)
    paste_data(ws, data)
    print("Done")
    

In [7]:
def relevant(i):
    return i.sprint_milestone == "Legacy" and i.status not in ("Closed", "Rejected")

trace_path = 'C:\\Users\\Nathan\\Documents\\Dropbox\\FRS'
user_tests = 'FRS Tests 181127.xlsx'
p1 = os.path.join(trace_path, user_tests)
main(p1, relevant)

Loading User Test Matrix...
Compiling data for final matrix
Pasting test data
Applying alignment formatting
Marking untested cells
Applying row formatting
Applying column autofit
Applying row autofit
Done


In [39]:
xl = Excel()
with screen_lock(xl):
    v = xl.Selection
    for row in v.Rows:
        h = row.Cells(1,1).Value
        if h != "Y":
            row.Cells(1, 3).Value = "'--"
            row.Cells(1, 3).HorizontalAlignment=xlc.xlCenter